![image.png](attachment:image.png)

##### **Good luck with taking your exam. Keep working and make your dreams all come true. Seeing the results of all of your hard work will make this struggle worth it. We’re all thinking of you.** 
<b><font color='blue'>AI-PRO Spark Team ITI</font></b>

# NLP Using PySpark

## Objective:
- The objective from this project is to create a <b>Spam filter using NaiveBayes classifier</b>.
- It is required to obtain <b>f1_scored > 0.9</b>.
- We'll use a dataset from UCI Repository. SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
- Data is also provided for you in the assignment (you do not have to download it).

## To perform this task follow the following guiding steps:

### Create a spark session and import the required libraries

In [88]:
pip install pyspark

In [89]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Final_exam").getOrCreate()

### Read the readme file to learn more about the data

### Read the data into a DataFrame

In [90]:
file_path = "SMSSpamCollection"
sample_df = spark.read.csv(file_path, header=False, inferSchema=True, samplingRatio=0.0001, sep="\t")
df_scema = sample_df.schema

In [91]:
sample_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



### Print the schema

In [92]:
df_bad_column_name = spark.read.csv(file_path, header=False, schema=df_scema, sep="\t")
df_bad_column_name.show(2)

+---+--------------------+
|_c0|                 _c1|
+---+--------------------+
|ham|Go until jurong p...|
|ham|Ok lar... Joking ...|
+---+--------------------+
only showing top 2 rows



### Rename the first column to 'class' and second column to 'text'

In [93]:
df = df_bad_column_name.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")

In [94]:
df.show(2)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
+-----+--------------------+
only showing top 2 rows



In [95]:
df.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)



### Show the first 10 rows from the dataframe
- Show once with truncate=True and once with truncate=False

In [96]:
df.show(10)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
+-----+--------------------+
only showing top 10 rows



In [97]:
df.show(10, truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                 |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                   |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075o

## Clean and Prepare the Data

### Create a new feature column contains the length of the text column

In [98]:
from pyspark.sql.functions import *

df2 = df.withColumn("length", length("text"))

### Show the new dataframe

In [99]:
df2.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



### Get the average text length for each class (give alias name to the average length column)

In [100]:
df2.groupby("class").agg(avg("length").alias("Avg. Lenght")).show()


+-----+-----------------+
|class|      Avg. Lenght|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

### In this part you transform you raw text in to tf_idf model :
- For more information about TF-IDF check the following link: <b>(Not needed for the test)</b>
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

### Perform the following steps to obtain TF-IDF:
1. Import the required transformers/estimators for the subsequent steps.
2. Create a <b>Tokenizer</b> from the text column.
3. Create a <b>StopWordsRemover</b> to remove the <b>stop words</b> from the column obtained from the <b>Tokenizer</b>.
4. Create a <b>CountVectorizer</b> after removing the <b>stop words</b>.
5. Create the <b>TF-IDF</b> from the <b>CountVectorizer</b>.

In [101]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, HashingTF, IDF, StringIndexer, VectorAssembler

In [102]:
# Create tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="words")

# Create StopWordsRemover
remover = StopWordsRemover(inputCol="words", outputCol="filtered")

# Create CountVectorizer
cv = CountVectorizer(inputCol="filtered", outputCol="counts")

# Create the TF-IDF 
## Create HashingTF
#hashingTF = HashingTF(inputCol="counts", outputCol="rawFeatures")

## Create IDF 
#idf = IDF(inputCol="rawFeatures", outputCol="idfFeatures")
idf = IDF(inputCol="counts", outputCol="idfFeatures")

- Convert the <b>class column</b> to index using <b>StringIndexer</b>
- Create feature column from the <b>TF-IDF</b> and <b>lenght</b> columns.

In [103]:
# create StringIndexer
indexer = StringIndexer(inputCol="class", outputCol="classIndex")

In [114]:
# create feature column from the TF-IDF and lenght columns.

# create VectorAssembler
#assembler2 = VectorAssembler(inputCols=["classIndex", "tf_and_length", "idfFeatures"], outputCol="features")
assembler = VectorAssembler(inputCols=["idfFeatures", "length"], outputCol="features")

In [113]:
df2.columns

['class', 'text', 'length']

## The Model
- Create a <b>NaiveBayes</b> classifier with the default parameters.

In [115]:
from pyspark.ml.classification import NaiveBayes
naivebayes = NaiveBayes(featuresCol="features", labelCol="classIndex")

## Pipeline
### Create a pipeline model contains all the steps starting from the Tokenizer to the NaiveBays classifier.

In [116]:
from pyspark.ml import Pipeline
pipeline_nb = Pipeline(stages=[tokenizer, remover, cv, idf, indexer, assembler, naivebayes])

### Split your data to trian and test data with ratios 0.7 and 0.3 respectively.

In [117]:
train_df, test_df = df2.randomSplit([0.7, 0.3], seed=42)

### Fit your Pipeline model to the training data

In [118]:
model = pipeline_nb.fit(train_df)

### Perform predictions on tests dataframe

In [119]:
predictions = model.transform(test_df)

### Print the schema of the prediction dataframe

In [120]:
predictions.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- counts: vector (nullable = true)
 |-- idfFeatures: vector (nullable = true)
 |-- classIndex: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)
 |-- label: double (nullable = false)
 |-- token_text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stop_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- c_vec: vector (nullable = true)
 |-- tf_idf: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



## Model Evaluation
- Use <b>MulticlassClassificationEvaluator</b> to calculate the <b>f1_score</b>.

In [124]:
predict_df = model.transform(test_df)

In [138]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="classIndex", predictionCol="prediction", metricName="f1")

In [140]:
score = evaluator.evaluate(predict_df.select("prediction", "classIndex"))

In [141]:
print("f1_score is: ", score)

f1_score is:  0.9727502290227267


# GOOD LUCK
<b><font color='GREEN'>AI-PRO Spark Team ITI</font></b>

![image-3.png](attachment:image-3.png)